# チャットボット作成  
https://www.udemy.com/course/ai-nlp-bot/  
(参照元)

チャットボット作成の流れは、  
①文字の読み込み  
②文章のベクトル化  
③返答用の関数  
④チャットボットの構築  
である。  


コーパスを作成する。  
青空文庫より、宮沢賢治の作品を元に作成する。

## ①文字の読み込みのための準備  
・青空文庫からデータを取得する。  
・テキストの量は少ないと精度が悪くなり、多いと学習に時間を要する。よって10個のテキストによって学習を行う。  

kenji_novelsというフォルダにテキストファイルを用意する。  
青空文庫URL  
https://www.aozora.gr.jp/index_pages/person81.html

In [ ]:
import re

novels = ["gingatetsudono_yoru.txt", "serohikino_goshu.txt", "chumonno_oi_ryoriten.txt",
         "gusukobudorino_denki.txt", "kaeruno_gomugutsu.txt", "kaino_hi.txt", "kashiwabayashino_yoru.txt",
         "kazeno_matasaburo.txt", "kiirono_tomato.txt", "oinomorito_zarumori.txt"]  # 青空文庫より

text = ""
for novel in novels:
    with open("kenji_novels/"+novel, mode="r", encoding="utf-8") as f:  # ファイルの読み込み
        text_novel = f.read()
    text_novel = re.sub("《[^》]+》", "", text_novel)  # ルビの削除
    text_novel = re.sub("［[^］]+］", "", text_novel)  # 読みの注意の削除
    text_novel = re.sub("〔[^〕]+〕", "", text_novel)  # 読みの注意の削除
    text_novel = re.sub("[ 　\n「」『』（）｜※＊…]", "", text_novel)  # 全角半角スペース、改行、その他記号の削除
    text += text_novel

In [ ]:
#テキストをひらがな、カタカナ、記号のみにする。
from pykakasi import kakasi

text = text.replace("苹果", "ひょうか")
#pykakasiに載っていない漢字なので手動で平仮名に変換する。

seperator = "。"  # 。をセパレータに指定
sentence_list = text.split(seperator)  # セパレーターを使って文章をリストに分割する
sentence_list.pop() # 最後の要素は空の文字列になるので、削除
sentence_list = [x+seperator for x in sentence_list]  # 文章の最後に。を追加

kakasi = kakasi()
kakasi.setMode("J", "H")  # J(漢字) からH(ひらがな)へ
conv = kakasi.getConverter()

kana_text = conv.do(text)

In [ ]:
#テキストをファイルに保存する。
with open("kana_kenji.txt", mode="w", encoding="utf-8") as f:
    f.write(kana_text)

In [ ]:
import pickle

hiragana = "ぁあぃいぅうぇえぉおかがきぎくぐけげこごさざしじすずせぜそぞ\
ただちぢっつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽ\
まみむめもゃやゅゆょよらりるれろゎわゐゑをん"

katakana = "ァアィイゥウェエォオカガキギクグケゲコゴサザシジスズセゼソゾ\
タダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポ\
マミムメモャヤュユョヨラリルレロヮワヰヱヲンヴ"

chars = hiragana + katakana

with open("kana_kenji.txt", mode="r", encoding="utf-8") as f:  # 直前で保存したファイル
    text = f.read()
    
for char in text:  # ひらがな、カタカナ以外でコーパスに使われている文字を追加
    if char not in chars:
        chars += char
        
chars += "\t\n"  # タブと改行を追加
        
chars_list = sorted(list(chars))  # 文字列をリストに変換してソートする

with open("kana_chars.pickle", mode="wb") as f:  # pickleで保存
    pickle.dump(chars_list, f)

In [ ]:
#kana_chars.pickleを作ったので、それを読み込む。
with open('kana_chars.pickle', mode='rb') as f:
    chars_list = pickle.load(f)

def is_invalid(message):
    is_invalid =False
    for char in message:
        if char not in chars_list:
            is_invalid = True
    return is_invalid

## ②one-hot表現による、文章のベクトル化を行う。

In [ ]:
import numpy as np

# インデックスと文字で辞書を作成
char_indices = {}
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}
for i, char in enumerate(chars_list):
    indices_char[i] = char
    
n_char = len(chars_list)
max_length_x = 128

# 文章をone-hot表現に変換する関数
def sentence_to_vector(sentence):
    vector = np.zeros((1, max_length_x, n_char), dtype=np.bool)
    for j, char in enumerate(sentence):
        vector[0][j][char_indices[char]] = 1
    return vector

## ③返答用の関数を設定する。

### (1)学習モデルの構築

In [ ]:
import numpy as np

# インデックスと文字で辞書を作成
char_indices = {}  # 文字がキーでインデックスが値
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}  # インデックスがキーで文字が値
for i, char in enumerate(chars_list):
    indices_char[i] = char
    
seperator = "。"
sentence_list = text.split(seperator) 
sentence_list.pop() 
sentence_list = [x+seperator for x in sentence_list]

max_sentence_length = 128  # 文章の最大長さ。これより長い文章はカットされる。
sentence_list = [sentence for sentence in sentence_list if len(sentence) <= max_sentence_length]  # 長すぎる文章のカット

n_char = len(chars_list)  # 文字の種類の数
n_sample = len(sentence_list) - 1  # サンプル数

x_sentences = []  # 入力の文章
t_sentences = []  # 正解の文章
for i in range(n_sample):
    x_sentences.append(sentence_list[i])
    t_sentences.append("\t" + sentence_list[i+1] + "\n")  # 正解は先頭にタブ、末尾に改行を加える
max_length_x = max_sentence_length  # 入力文章の最大長さ
max_length_t = max_sentence_length + 2  # 正解文章の最大長さ

x_encoder = np.zeros((n_sample, max_length_x, n_char), dtype=np.bool)  # encoderへの入力
x_decoder = np.zeros((n_sample, max_length_t, n_char), dtype=np.bool)  # decoderへの入力
t_decoder = np.zeros((n_sample, max_length_t, n_char), dtype=np.bool)  # decoderの正解

for i in range(n_sample):
    x_sentence = x_sentences[i]
    t_sentence = t_sentences[i]
    for j, char in enumerate(x_sentence):
        x_encoder[i, j, char_indices[char]] = 1  # encoderへの入力をone-hot表現で表す
    for j, char in enumerate(t_sentence):
        x_decoder[i, j, char_indices[char]] = 1  # decoderへの入力をone-hot表現で表す
        if j > 0:  # 正解は入力より1つ前の時刻のものにする
            t_decoder[i, j-1, char_indices[char]] = 1
            
print(x_encoder.shape)

In [ ]:
from keras.models import Model
from keras.layers import Dense, GRU, Input, Masking

batch_size = 32
epochs = 1000
n_mid = 256  # 中間層のニューロン数

encoder_input = Input(shape=(None, n_char))
encoder_mask = Masking(mask_value=0)  # 全ての要素が0であるベクトルの入力は無視する
encoder_masked = encoder_mask(encoder_input)
encoder_lstm = GRU(n_mid, dropout=0.2, recurrent_dropout=0.2, return_state=True)  # dropoutを設定し、ニューロンをランダムに無効にする
encoder_output, encoder_state_h = encoder_lstm(encoder_masked)

decoder_input = Input(shape=(None, n_char))
decoder_mask = Masking(mask_value=0)  # 全ての要素が0であるベクトルの入力は無視する
decoder_masked = decoder_mask(decoder_input)
decoder_lstm = GRU(n_mid, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=True)  # dropoutを設定
decoder_output, _ = decoder_lstm(decoder_masked, initial_state=encoder_state_h)  # encoderの状態を初期状態にする
decoder_dense = Dense(n_char, activation='softmax')
decoder_output = decoder_dense(decoder_output)

model = Model([encoder_input, decoder_input], decoder_output)

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [ ]:
from keras.callbacks import EarlyStopping 

#val_lossに改善が見られなくなってから、30エポックで学習は終了
#学習に時間を要するので、終了するエポック数は減らしても良いかもしれない。
#下記のコードでは124回行って終了した。
early_stopping = EarlyStopping(monitor="val_loss", patience=30) 

history = model.fit([x_encoder, x_decoder], t_decoder,
                     batch_size=batch_size,
                     epochs=epochs,
                     validation_split=0.1,  # 10%は検証用
                     callbacks=[early_stopping])

### (2)予測用モデルの構築

In [ ]:
# encoderのモデル
encoder_model = Model(encoder_input, encoder_state_h)

# decoderのモデル
decoder_state_in_h = Input(shape=(n_mid,))
decoder_state_in = [decoder_state_in_h]

decoder_output, decoder_state_h = decoder_lstm(decoder_input,
                                               initial_state=decoder_state_in_h)
decoder_output = decoder_dense(decoder_output)

decoder_model = Model([decoder_input] + decoder_state_in,
                      [decoder_output, decoder_state_h])

# モデルの保存
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')

### (3)返答用関数の設定

In [ ]:
from keras.models import load_model

encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')

def respond(message, beta=5):
    vec = sentence_to_vector(message)  # 文字列をone-hot表現に変換
    state_value = encoder_model.predict(vec)
    y_decoder = np.zeros((1, 1, n_char))  # decoderの出力を格納する配列
    y_decoder[0][0][char_indices['\t']] = 1  # decoderの最初の入力はタブ。one-hot表現にする。

    respond_sentence = ""  # 返答の文字列
    while True:
        y, h = decoder_model.predict([y_decoder, state_value])
        p_power = y[0][0] ** beta  # 確率分布の調整
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power)) 
        next_char = indices_char[next_index]  # 次の文字
        
        if (next_char == "\n" or len(respond_sentence) >= max_length_x):
            break  # 次の文字が改行のとき、もしくは最大文字数を超えたときは終了
            
        respond_sentence += next_char
        y_decoder = np.zeros((1, 1, n_char))  # 次の時刻の入力
        y_decoder[0][0][next_index] = 1

        state_value = h  # 次の時刻の状態

    return respond_sentence

## ④チャットボットの構築を行う。

In [ ]:
bot_name = "賢治bot"
your_name = input("おなまえをおしえてください。:")
print()

print(bot_name + ": " + "こんにちは、" + your_name + "さん。")
message = ""
while message != "さようなら。":
    
    while True:
        message = input(your_name + ": ")
        if not is_invalid(message):
            break
        else:
            print(bot_name + ": ひらがなか、カタカナをつかってください。")
            
    response = respond(message)
    print(bot_name + ": " + response)

(課題)  
返答の精度が悪い点。  
これには、コーパスをかえる